In [1]:
%pylab inline

from __future__ import division
import numpy as np
import pandas as pd
from lasagne import layers
from nolearn.lasagne import NeuralNet, TrainSplit, BatchIterator
from lasagne.updates import adadelta, adam, nesterov_momentum
from lasagne.nonlinearities import very_leaky_rectify
from scipy.stats import spearmanr
from sklearn.utils import shuffle
from random import choice, sample
import cPickle as pickle
import theano
import os
import gc

def plot_results(net, y1, y2, x2): 
    train_loss = np.array([i["train_loss"] for i in net.train_history_])
    valid_loss = np.array([i["valid_loss"] for i in net.train_history_])
    pyplot.plot(train_loss, linewidth=3, label="train")
    pyplot.plot(valid_loss, linewidth=3, label="valid")
    pyplot.grid()
    pyplot.legend()
    pyplot.xlabel("epoch")
    pyplot.ylabel("loss")
    pyplot.ylim(y1, y2)
    pyplot.xlim(-50, x2)
    pyplot.show()
    
            
def mean_cor(y, yhat):
    nrows = y.shape[0]
    cors = []
    
    # get correlation between each predicted and actual sample
    for i in range(nrows):
        cors.append(spearmanr(y[i, :], yhat[i, :])[0])
        
    return np.mean(cors)

def accuracy(y, yhat):
    num_equal = np.sum(np.sign(y) == np.sign(yhat))
    num_total = y.shape[0] * y.shape[1]
    
    return(num_equal / num_total)

# accuracy as a fraction of average model
def scaled_accuracy(y, yhat):
    return (accuracy(y, yhat) / 0.789634)


def scaled_cor(y, yhat):
    cor = mean_cor(y, yhat)   
    return (mean_cor(y, yhat) / 0.57595569)


def mae(y, yhat):
    return np.mean(np.absolute(y-yhat))



class FlipBatchIterator(BatchIterator):

    def transform(self, Xb, yb):
        Xb, yb = super(FlipBatchIterator, self).transform(Xb, yb)

        # number of samples and genes
        ns = Xb.shape[0]
        ng = Xb.shape[1] // 2
        
        # for random half of genes and samples, swap d1 and d2
        sw = [choice([0,1]) for _ in range(ng)]
        
        a = [ i    +(s*ng) for i, s in zip(range(ng), sw)]
        b = [(i+ng)-(s*ng) for i, s in zip(range(ng), sw)]

        cols  = a + b
        rows  = sample(range(ns), ns // 2)
        
        ind_orig = np.ix_(rows, range(ng*2))
        ind_flip = np.ix_(rows, cols)
        
        Xb[ind_orig] = Xb[ind_flip]
        
        return Xb, yb
    
def float32(k):
    return np.cast['float32'](k)

class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)
        
class PickleModel(object):
    def __init__(self, freq=10):
        self.freq = freq

    def __call__(self, nn, train_history):
        epoch = train_history[-1]['epoch']
        
        if epoch % self.freq == 0:
            with open('net_int.pickle', 'wb') as f:
                pickle.dump(nn, f, -1)

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled)
/home/alex/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


Populating the interactive namespace from numpy and matplotlib


In [2]:
# load data
X = np.load('data/X.npy')
y = np.load('data/y.npy')

# X = X.astype(np.float32)
# y = y.astype(np.float32)
# np.save('data/X', X)
# np.save('data/y', y)

In [3]:
# accuracy from average model
avg = (X[:, :11525] + X[:, 11525:]) / 2
print 'avg accuracy: ', accuracy(y, avg)

# scaled to average model (def as 1)
# scaled_accuracy(avg, y)

# mean spearman correlation from average model
print 'avg spearmanr:', mean_cor(avg, y)

# mean spearman correlation of absolute values from average model
print 'abs spearmanr:', mean_cor(np.absolute(avg), np.absolute(y))

avg accuracy:  0.789634443816
avg spearmanr: 0.760325450471
abs spearmanr: 0.575955691642


In [22]:
# shuffle data
ids = shuffle(range(y.shape[0]), random_state=0)

# split data in half
X1 = X[ids[:128]]
X2 = X[ids[128:]]

y1 = y[ids[:128]]
y2 = y[ids[128:]]

X   = X[ids]
y   = y[ids]


gc.collect()

16

<hr />
## Configuration:
<hr />

* <b>hidden layers:</b>
* <b>dropout:</b>
* <b>epochs:</b>

<b>Summary of Changes:</b> <br>


    
<hr />

In [4]:
#train model on first half of data
net1 = NeuralNet(
    layers=[
        ('input',   layers.InputLayer),
        ('dropout1', layers.DropoutLayer),
        ('hidden',  layers.DenseLayer),
        ('dropout2', layers.DropoutLayer),
        ('output',  layers.DenseLayer),
        ],
    # layer parameters:
    input_shape         = (None, X.shape[1]),
    dropout1_p          = 0.85,
    dropout2_p          = 0.5,
    hidden_num_units    = 2500,
    hidden_nonlinearity = very_leaky_rectify,
    output_nonlinearity = None, 
    output_num_units    = y.shape[1],

    # optimization method:
    batch_iterator_train    = FlipBatchIterator(batch_size=70),
    train_split             = TrainSplit(eval_size=0.0),
    update                  = nesterov_momentum,
    update_learning_rate    = theano.shared(float32(0.01)),
    update_momentum         = theano.shared(float32(0.9)),
    regression              = True, 
    max_epochs              = 5000,
    verbose                 = 1,
    on_epoch_finished       = [AdjustVariable('update_learning_rate', start=0.01, stop=0.00001),
                               AdjustVariable('update_momentum', start=0.9, stop=0.999)],
    )

np.random.seed(0)
net1.fit(X1, y1)

# Neural Network with 86451525 learnable parameters

## Layer information

  #  name        size
---  --------  ------
  0  input      23050
  1  dropout1   23050
  2  hidden      2500
  3  dropout2    2500
  4  output     11525

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1      28.79941           nan          nan  0.17s
      2      27.20543           nan          nan  0.12s
      3      25.46894           nan          nan  0.12s
      4      23.93283           nan          nan  0.11s
      5      22.63622           nan          nan  0.11s
      6      22.33615           nan          nan  0.11s
      7      21.61326           nan          nan  0.11s
      8      21.42779           nan          nan  0.11s
      9      20.93595           nan          nan  0.11s
     10      20.23365           nan          nan  0.11s
     11      20.12444           nan          nan  0.11s
     12      19.82211           nan         

/home/alex/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f7ca92754d0>,
     batch_iterator_train=<__main__.FlipBatchIterator object at 0x7f7cc886c850>,
     custom_scores=None, dropout1_p=0.85, dropout2_p=0.5,
     hidden_nonlinearity=<lasagne.nonlinearities.LeakyRectify object at 0x7f7caa885fd0>,
     hidden_num_units=2500, input_shape=(None, 23050),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('dropout1', <class 'lasagne.layers.noise.DropoutLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout2', <class 'lasagne.layers.noise.DropoutLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=5000, more_params={},
     objective=<function objective at 0x7f7ca9277c80>,
     objective_loss_function=<function squared_error at 0x7f7caa449938>,
     on_batch_finished=[],
     on_epoch_finished=[<__main__.AdjustVariable object at 0x7f7ce446ac50>, <__main__.AdjustVariabl

In [6]:
# save model
with open('net1.pickle', 'wb') as f:
    pickle.dump(net1, f, -1)

In [5]:
#train model on second half of data
net2 = NeuralNet(
    layers=[
        ('input',   layers.InputLayer),
        ('dropout1', layers.DropoutLayer),
        ('hidden',  layers.DenseLayer),
        ('dropout2', layers.DropoutLayer),
        ('output',  layers.DenseLayer),
        ],
    # layer parameters:
    input_shape         = (None, X.shape[1]),
    dropout1_p          = 0.85,
    dropout2_p          = 0.5,
    hidden_num_units    = 2500,
    hidden_nonlinearity = very_leaky_rectify,
    output_nonlinearity = None, 
    output_num_units    = y.shape[1],

    # optimization method:
    batch_iterator_train    = FlipBatchIterator(batch_size=70),
    train_split             = TrainSplit(eval_size=0.0),
    update                  = nesterov_momentum,
    update_learning_rate    = theano.shared(float32(0.01)),
    update_momentum         = theano.shared(float32(0.9)),
    regression              = True, 
    max_epochs              = 5000,
    verbose                 = 1,
    on_epoch_finished       = [AdjustVariable('update_learning_rate', start=0.01, stop=0.00001),
                               AdjustVariable('update_momentum', start=0.9, stop=0.999)],
    )

np.random.seed(0)
net2.fit(X2, y2)

# Neural Network with 86451525 learnable parameters

## Layer information

  #  name        size
---  --------  ------
  0  input      23050
  1  dropout1   23050
  2  hidden      2500
  3  dropout2    2500
  4  output     11525

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1      27.70298           nan          nan  0.11s
      2      26.19698           nan          nan  0.11s
      3      24.86581           nan          nan  0.11s
      4      23.14358           nan          nan  0.11s
      5      22.25218           nan          nan  0.11s
      6      21.41219           nan          nan  0.11s
      7      20.81907           nan          nan  0.11s
      8      20.44415           nan          nan  0.11s
      9      19.99846           nan          nan  0.11s
     10      19.42081           nan          nan  0.11s
     11      19.13257           nan          nan  0.11s
     12      18.64470           nan         

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f7ca92754d0>,
     batch_iterator_train=<__main__.FlipBatchIterator object at 0x7f7c9d5dc150>,
     custom_scores=None, dropout1_p=0.85, dropout2_p=0.5,
     hidden_nonlinearity=<lasagne.nonlinearities.LeakyRectify object at 0x7f7caa885fd0>,
     hidden_num_units=2500, input_shape=(None, 23050),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('dropout1', <class 'lasagne.layers.noise.DropoutLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout2', <class 'lasagne.layers.noise.DropoutLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=5000, more_params={},
     objective=<function objective at 0x7f7ca9277c80>,
     objective_loss_function=<function squared_error at 0x7f7caa449938>,
     on_batch_finished=[],
     on_epoch_finished=[<__main__.AdjustVariable object at 0x7f7c9d5dcc10>, <__main__.AdjustVariabl

In [7]:
# save model
with open('net2.pickle', 'wb') as f:
    pickle.dump(net2, f, -1)

In [9]:
# open model
with open('data/net1/net1.pickle', 'rb') as f:
    net1 = pickle.load(f)
    
with open('data/net2/net2.pickle', 'rb') as f:
    net2 = pickle.load(f)

In [117]:
# reload original data (X1 and X2 gene values swapped in place)
X = np.load('data/X.npy')
y = np.load('data/y.npy')

Xv = np.load('data/Xv.npy')
yv = np.load('data/yv.npy')

# shuffle data
ids = shuffle(range(y.shape[0]), random_state=0)

# split data in half
X1 = X[ids[:128]]
X2 = X[ids[128:]]

y1 = y[ids[:128]]
y2 = y[ids[128:]]

X   = X[ids]
y   = y[ids]


# same for variances
Xv1 = Xv[ids[:128]]
Xv2 = Xv[ids[128:]]

yv1 = yv[ids[:128]]
yv2 = yv[ids[128:]]

Xv   = Xv[ids]
yv   = yv[ids]


gc.collect()

# predict y2 from X2 using mod1 (trained on y1/X1)
y2_preds = net1.predict(X2)

# predict y1 from X1 using mod2 (trained on y2/X2)
y1_preds = net2.predict(X1)

In [12]:
# compare predictions to truth
print "net1 accuracy:", accuracy(y2, y2_preds)
print "net2 accuracy:", accuracy(y1, y1_preds), "\n"

print "net1 mae:", mae(y2, y2_preds)
print "net2 mae:", mae(y1, y1_preds), "\n"

print "net1 spearmanr:", mean_cor(y2, y2_preds)
print "net2 spearmanr:", mean_cor(y1, y1_preds)

net1 accuracy: 0.682884864383
net2 accuracy: 0.680730070499 

net1 mae: 1.20303
net2 mae: 1.25957 

net1 spearmanr: 0.492896164695
net2 spearmanr: 0.498007191433


In [118]:
# test reshaping
pf1 = np.array([['cbo_s1g1','cbo_s1g2','cbo_s1g3','cbo_s1g4'],
                ['cbo_s2g1','cbo_s2g2','cbo_s2g3','cbo_s2g4']])

pf2 = np.array([['cbo_s3g1','cbo_s3g2','cbo_s3g3','cbo_s3g4'],
                ['cbo_s4g1','cbo_s4g2','cbo_s4g3','cbo_s4g4']])

Xf = np.array([['d1_s1g1','d1_s1g2','d1_s1g3','d1_s1g4','d2_s1g1','d2_s1g2','d2_s1g3','d2_s1g4'],
               ['d1_s2g1','d1_s2g2','d1_s2g3','d1_s2g4','d2_s2g1','d2_s2g2','d2_s2g3','d2_s2g4'],
               ['d1_s3g1','d1_s3g2','d1_s3g3','d1_s3g4','d2_s3g1','d2_s3g2','d2_s3g3','d2_s3g4'],
               ['d1_s4g1','d1_s4g2','d1_s4g3','d1_s4g4','d2_s4g1','d2_s4g2','d2_s4g3','d2_s4g4']])

yf = np.array([['y_s1g1','y_s1g2','y_s1g3','y_s1g4'],
               ['y_s2g1','y_s2g2','y_s2g3','y_s2g4'],
               ['y_s3g1','y_s3g2','y_s3g3','y_s3g4'],
               ['y_s4g1','y_s4g2','y_s4g3','y_s4g4']])


In [119]:
# stack preds for each sample (rows) on top of each other
pf = reshape(np.vstack((pf1, pf2)), -1, 'A')

# stack samples on top of each other with d1 and d2 for each gene side by side
Xf  = np.transpose(np.vstack((reshape(Xf[:,:4], -1, 'A'), reshape(Xf[:,4:], -1, 'A'))))

yf  = np.reshape(yf,  -1, 'A')

# concatenate preds, X, and y
np.c_[pf, Xf, yf]

array([['cbo_s1g1', 'd1_s1g1', 'd2_s1g1', 'y_s1g1'],
       ['cbo_s1g2', 'd1_s1g2', 'd2_s1g2', 'y_s1g2'],
       ['cbo_s1g3', 'd1_s1g3', 'd2_s1g3', 'y_s1g3'],
       ['cbo_s1g4', 'd1_s1g4', 'd2_s1g4', 'y_s1g4'],
       ['cbo_s2g1', 'd1_s2g1', 'd2_s2g1', 'y_s2g1'],
       ['cbo_s2g2', 'd1_s2g2', 'd2_s2g2', 'y_s2g2'],
       ['cbo_s2g3', 'd1_s2g3', 'd2_s2g3', 'y_s2g3'],
       ['cbo_s2g4', 'd1_s2g4', 'd2_s2g4', 'y_s2g4'],
       ['cbo_s3g1', 'd1_s3g1', 'd2_s3g1', 'y_s3g1'],
       ['cbo_s3g2', 'd1_s3g2', 'd2_s3g2', 'y_s3g2'],
       ['cbo_s3g3', 'd1_s3g3', 'd2_s3g3', 'y_s3g3'],
       ['cbo_s3g4', 'd1_s3g4', 'd2_s3g4', 'y_s3g4'],
       ['cbo_s4g1', 'd1_s4g1', 'd2_s4g1', 'y_s4g1'],
       ['cbo_s4g2', 'd1_s4g2', 'd2_s4g2', 'y_s4g2'],
       ['cbo_s4g3', 'd1_s4g3', 'd2_s4g3', 'y_s4g3'],
       ['cbo_s4g4', 'd1_s4g4', 'd2_s4g4', 'y_s4g4']], 
      dtype='|S8')

In [120]:
# stack preds for each sample (rows) on top of each other
preds = reshape(np.vstack((y1_preds, y2_preds)), -1, 'A')

# stack samples on top of each other with d1 and d2 for each gene side by side
X  = np.transpose(np.vstack((reshape(X[:,:11525],  -1, 'A'), reshape(X[:,11525:],  -1, 'A'))))
Xv = np.transpose(np.vstack((reshape(Xv[:,:11525], -1, 'A'), reshape(Xv[:,11525:], -1, 'A'))))

y  = np.reshape(y,  -1, 'A')
yv = np.reshape(yv, -1, 'A')

# concatenate preds, X, and y
train = np.c_[preds, X, Xv, y]

# save result for xgboost training
np.savetxt("data/stack/train.csv", train, delimiter=",")

In [121]:
train.shape

(2961925, 6)

In [416]:
# net1 parameters
net1_W1 = net1.get_all_params_values()['hidden'][0]
net1_W2 = net1.get_all_params_values()['output'][0]

net1_b1 = net1.get_all_params_values()['hidden'][1]
net1_b2 = net1.get_all_params_values()['output'][1]

# net2 parameters
net2_W1 = net2.get_all_params_values()['hidden'][0]
net2_W2 = net2.get_all_params_values()['output'][0]

net2_b1 = net2.get_all_params_values()['hidden'][1]
net2_b2 = net2.get_all_params_values()['output'][1]

In [444]:
# predict function
def pred(W1, W2, b1, b2, X):
    z2 = np.dot(X, W1) + b1
    a2 = np.maximum(z2/3, z2)
    return np.dot(a2, W2) + b2

In [415]:
pd.DataFrame(net1_W1).to_csv("data/net1/W1.csv", header=False, index=False)
pd.DataFrame(net1_W2).to_csv("data/net1/W2.csv", header=False, index=False)
pd.DataFrame(net1_q1).to_csv("data/net1/b1.csv", header=False, index=False)
pd.DataFrame(net1_q2).to_csv("data/net1/b2.csv", header=False, index=False)

In [454]:
pd.DataFrame(net2_W1).to_csv("data/net2/W1.csv", header=False, index=False)
pd.DataFrame(net2_W2).to_csv("data/net2/W2.csv", header=False, index=False)
pd.DataFrame(net2_q1).to_csv("data/net2/b1.csv", header=False, index=False)
pd.DataFrame(net2_q2).to_csv("data/net2/b2.csv", header=False, index=False)